In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures, OrdinalEncoder, OneHotEncoder, StandardScaler, RobustScaler, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score,make_scorer, accuracy_score


### LES FEATURES A GARDER SUITE A L'EDA

## Analyse du CHI2
State, NAICS(ou le cat_activites), UrbanRural, LowDoc, Bank,RevLineCr,FranchiseCode

## Analyse de l'ANOVA
DisbursementGross,bank_loan_float(float de GrAppv), SBA_loan_float(SBA_Appv)

In [59]:
df = pd.read_csv('dataset.csv')
df['NewExist'] = df['NewExist'].fillna(0)
df.columns

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv',
       'cat_activites', 'SBA_loan_float', 'bank_loan_float', 'crisis'],
      dtype='object')

In [73]:
features_of_interest = [
    'State','cat_activites', 'UrbanRural','LowDoc','bank_loan_float','SBA_loan_float','FranchiseCode','crisis','BankState','RevLineCr',
    'MIS_Status'
]

numerical_column = ['bank_loan_float','SBA_loan_float',
    
]


ordinal_column = [
    'LowDoc'
]

categorical_column = [
    'State','cat_activites','FranchiseCode','BankState','RevLineCr'
]


target_name = "MIS_Status"
data, target, numerical_data, ordinal_data, categorical_data = (
    df[features_of_interest],
    df[target_name],
    df[numerical_column],
    df[ordinal_column],
    df[categorical_column]
)

In [16]:
data.isna().sum()

State                 0
cat_activites         0
UrbanRural            0
LowDoc                0
bank_loan_float       0
SBA_loan_float        0
FranchiseCode         0
crisis                0
Bank               1559
NoEmp                 0
NewExist              0
CreateJob             0
RetainedJob           0
MIS_Status            0
dtype: int64

In [74]:
X_train, X_test, y_train, y_test = train_test_split(data, target,train_size=0.9, random_state=42, stratify=data['MIS_Status'])
# Retirer la colonne 'MIS_Status' des ensembles X_train et X_test (car elle ne fait pas partie des features)
X_train = X_train.drop('MIS_Status', axis=1)
X_test = X_test.drop('MIS_Status', axis=1)


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

### Regression logistique

In [75]:
preprocessor = ColumnTransformer(
    [("categorical", OneHotEncoder(handle_unknown='ignore'), categorical_column),
    ("ordinal", OrdinalEncoder(), ordinal_column),
    ("numeric", StandardScaler(), numerical_column),
    ],
    remainder="passthrough",
)

log_model = make_pipeline(preprocessor,LogisticRegression())
log_model.fit(X_train,y_train_encoded)


/home/zerrouk/Documents/Simplon/brief_pret/brief_loan/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/zerrouk/Documents/Simplon/brief_pret/brief_loan/.venv/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in t

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['State', 'cat_activites',
                                                   'FranchiseCode', 'BankState',
                                                   'RevLineCr']),
                                                 ('ordinal', OrdinalEncoder(),
                                                  ['LowDoc']),
                                                 ('numeric', StandardScaler(),
                                                  ['bank_loan_float',
                                                   'SBA_loan_float'])])),
                ('logisticregression', LogisticRegression())])

In [56]:
log_model.predict_log_proba

<bound method Pipeline.predict_log_proba of Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['State', 'cat_activites',
                                                   'FranchiseCode', 'Bank',
                                                   'RevLineCr']),
                                                 ('ordinal', OrdinalEncoder(),
                                                  ['LowDoc']),
                                                 ('numeric', StandardScaler(),
                                                  ['bank_loan_float',
                                                   'SBA_loan_float'])])),
                ('logisticregression', LogisticRegression())])>

In [76]:
# Prédictions sur les scores de décision
decision_scores = log_model.decision_function(X_test)

# Calcul du score AUC-ROC
roc_auc = roc_auc_score(y_test, decision_scores)
acc_test = log_model.score(X_test,y_test_encoded)

acc_train = log_model.score(X_train,y_train_encoded)
print("AUC-ROC score:", roc_auc)
print("Accuracy sur le test:", acc_test)
print("Accuracy sur le train:", acc_train)

AUC-ROC score: 0.7548506811422855
Accuracy sur le test: 0.8327902398878966
Accuracy sur le train: 0.8329323432771453


In [116]:
df['Zip'].unique()

array([47711, 46526, 47401, ..., 70036, 66549, 26134], shape=(33611,))

### RANDOM FOREST GUMP

In [25]:
features_of_interest = [
    'State','cat_activites', 'UrbanRural','LowDoc','bank_loan_float','SBA_loan_float','FranchiseCode', 'crisis','Bank','MIS_Status'
]

numerical_column = ['bank_loan_float','SBA_loan_float'
]


ordinal_column = [
    'LowDoc'
]

categorical_column = [
    'State','cat_activites','FranchiseCode', 'Bank'
]


target_name = "MIS_Status"
data, target, numerical_data, ordinal_data, categorical_data = (
    df[features_of_interest],
    df[target_name],
    df[numerical_column],
    df[ordinal_column],
    df[categorical_column]
)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(data, target,train_size=0.9, random_state=42, stratify=data['MIS_Status'])
# Retirer la colonne 'MIS_Status' des ensembles X_train et X_test (car elle ne fait pas partie des features)
X_train = X_train.drop('MIS_Status', axis=1)
X_test = X_test.drop('MIS_Status', axis=1)


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [27]:
preprocessor = ColumnTransformer(
    [("categorical", OneHotEncoder(handle_unknown='ignore'), categorical_column),
    ("ordinal", OrdinalEncoder(), ordinal_column),
    ("numeric", RobustScaler(), numerical_column),
    ],
    remainder="passthrough",
)

rf_model = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=10))
rf_model.fit(X_train,y_train_encoded)

/home/zerrouk/Documents/Simplon/brief_pret/brief_loan/.venv/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['State', 'cat_activites',
                                                   'FranchiseCode', 'Bank']),
                                                 ('ordinal', OrdinalEncoder(),
                                                  ['LowDoc']),
                                                 ('numeric', RobustScaler(),
                                                  ['bank_loan_float',
                                                   'SBA_loan_float'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=10))])

In [36]:
# Si c'est un problème binaire, prenez la probabilité pour la classe 1 (classe positive)
proba = rf_model.predict_proba(X_test)[:, 1]

# Calcul du score AUC-ROC
roc_auc = roc_auc_score(y_test, proba)
print("AUC-ROC score:", roc_auc)
print(rf_model.score(X_test,y_test_encoded))

AUC-ROC score: 0.7532115902500633
0.821513173259784


In [45]:
np.sort(rf_model[-1].feature_importances_)

array([0.        , 0.        , 0.        , ..., 0.03896218, 0.19216009,
       0.20417527], shape=(8455,))

In [72]:
pd.DataFrame(rf_model.feature_names_in_, rf_model[-1].feature_importances_)

ValueError: Shape of passed values is (9, 1), indices imply (8455, 1)